In [1]:
import cv2
import pandas as pd
import numpy as np
import os
import csv
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
rows = []
imgs = []
for root, dirs, files in os.walk("input"):
    for name in files:
        path = os.path.join(root, name)
        img= cv2.imread(path)
        rows.append(path.split('_'))
        imgs.append(img)

# Output Dataset

In [3]:
col = [f"Y{i}" for i in range(12)]
col

['Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9', 'Y10', 'Y11']

In [4]:
data = pd.DataFrame(rows, columns=col)
data[["Y11","Y12"]]= data["Y11"].str.split(".", expand=True)
data = data.drop(['Y0', 'Y12'], axis=1)
data

,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11
0,4.0,4.02,2.54,0.59,0.59,119.37,116.8,65.54,10.0,100,10
1,3.96,3.96,2.54,0.6,0.6,121.43,121.57,63.32,10.0,100,12
2,4.07,4.07,2.51,0.59,0.59,114.64,116.35,65.82,10.0,10,10
3,4.01,4.06,2.48,0.59,0.59,118.76,115.7,65.57,10.0,10,12
4,4.07,4.03,2.58,0.6,0.6,119.31,122.56,63.81,10.0,10,8
...,...,...,...,...,...,...,...,...,...,...,...
1480,12.0,12.3,8.28,0.48,0.48,137.79,137.45,70.97,60.0,99,10
1481,12.48,12.61,8.74,0.48,0.48,148.18,147.91,76.82,60.0,99,8
1482,11.7,12.16,8.47,0.48,0.48,144.53,142.14,77.85,60.0,9,10
1483,12.15,11.91,8.04,0.48,0.48,142.57,145.6,71.21,60.0,9,12


In [5]:
data.dtypes

Y1     object
Y2     object
Y3     object
Y4     object
Y5     object
Y6     object
Y7     object
Y8     object
Y9     object
Y10    object
Y11    object
dtype: object

In [6]:
output_data = data.apply(pd.to_numeric)

In [7]:
output_data

,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,Y11
0,4.00,4.02,2.54,0.59,0.59,119.37,116.80,65.54,10.0,100,10
1,3.96,3.96,2.54,0.60,0.60,121.43,121.57,63.32,10.0,100,12
2,4.07,4.07,2.51,0.59,0.59,114.64,116.35,65.82,10.0,10,10
3,4.01,4.06,2.48,0.59,0.59,118.76,115.70,65.57,10.0,10,12
4,4.07,4.03,2.58,0.60,0.60,119.31,122.56,63.81,10.0,10,8
...,...,...,...,...,...,...,...,...,...,...,...
1480,12.00,12.30,8.28,0.48,0.48,137.79,137.45,70.97,60.0,99,10
1481,12.48,12.61,8.74,0.48,0.48,148.18,147.91,76.82,60.0,99,8
1482,11.70,12.16,8.47,0.48,0.48,144.53,142.14,77.85,60.0,9,10
1483,12.15,11.91,8.04,0.48,0.48,142.57,145.60,71.21,60.0,9,12


In [8]:
output_data.dtypes

Y1     float64
Y2     float64
Y3     float64
Y4     float64
Y5     float64
Y6     float64
Y7     float64
Y8     float64
Y9     float64
Y10      int64
Y11      int64
dtype: object

In [9]:
output_data.to_csv('Y columns.csv', index=False)

In [10]:
img.size, img.shape, img.ndim

(410700, (370, 370, 3), 3)

# Input Dataset

In [11]:
np_data = np.array(imgs)
print(np_data.shape)

(1485, 370, 370, 3)


In [15]:
df=pd.DataFrame(np_data)
df

ValueError: Must pass 2-d input. shape=(1485, 370, 370, 3)

In [12]:
np_data = np_data.reshape(1485, -1)

In [13]:
df=pd.DataFrame(np_data, columns=[f"Col{i}" for i in range(410700)])
df

,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,...,Col410690,Col410691,Col410692,Col410693,Col410694,Col410695,Col410696,Col410697,Col410698,Col410699
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1481,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.to_csv('input columns.csv', index=False)

### Data Normalization

In [15]:
scaler = MinMaxScaler()
input_data = scaler.fit_transform(df)

In [16]:
df1=pd.DataFrame(input_data, columns=[f"X{i}" for i in range(410700)])
df1

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X410690,X410691,X410692,X410693,X410694,X410695,X410696,X410697,X410698,X410699
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1481,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df1.to_csv('X columns.csv', index=False)

### Model Building

In [18]:
# X_train, X_rem, y_train, y_rem = train_test_split(input_data,output_data['C1'], train_size=0.8)
# X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [19]:
# X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape

In [20]:
# model = Sequential(name='CNN_Model')
# model.add(Conv1D(4, 2, activation='relu', input_shape=(410700,1)))
# model.add(MaxPool1D(2))
    
# model.add(Flatten())
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='softmax'))
# model.compile(optimizer=Adam(),loss = tf.keras.losses.MeanAbsolutePercentageError(), metrics= ['mse'])
# model.summary()

In [21]:
# cnn_history = model.fit(X_train, y_train, batch_size=10, epochs = 5, validation_data=(X_valid,y_valid),verbose=1)

In [22]:
# model.save('model.h5')